# Рейтинг на основании содержания Фильмов

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import mean_squared_error
from math import sqrt

# Загрузим данные
movies = pd.read_csv('../../src/MovieLens/movies.csv')
ratings = pd.read_csv('../../src/MovieLens/ratings.csv')
tags = pd.read_csv('../../src/MovieLens/tags.csv')

## собираем общий массив для `TF-DF`

In [3]:
ratings.rating = ratings.rating.fillna(0)
mean_ratings = ratings.groupby('movieId').agg({'rating':'mean'}).reset_index()
mean_ratings = mean_ratings.rename(columns={'rating': 'mean_rating'})

grouped_movies = movies.groupby('movieId').agg({
    'title': 'first',
    'genres': '|'.join,
}).reset_index()
len(grouped_movies) == len(movies)
grouped_movies.head()
tags['tag'] = tags['tag'].fillna('')
grouped_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
grouped_tags.head()

move_genres_and_tag = pd.merge(grouped_movies, grouped_tags, how='left').fillna('NoTag')

data = pd.merge(move_genres_and_tag, mean_ratings, how='left')
data['mean_rating'] = data['mean_rating'].fillna(0)

## `TF-DF` для `teg` и `genres`

In [4]:
tfidf_vectorizer_tags = TfidfVectorizer(stop_words='english')
tags_tfidf = tfidf_vectorizer_tags.fit_transform(data['tag'])


tfidf_vectorizer_genres = TfidfVectorizer(stop_words='english')
genres_tfidf = tfidf_vectorizer_genres.fit_transform(data['genres'])


## Объедениям и обучаем модель

In [5]:
tags_df = pd.DataFrame(tags_tfidf.toarray(), index=data['movieId']).reset_index()
genres_df = pd.DataFrame(genres_tfidf.toarray(), index=data['movieId']).reset_index()
data = pd.merge(data, tags_df, left_on='movieId', right_on='movieId', how='left')
data = pd.merge(data, genres_df, left_on='movieId', right_on='movieId', how='left')

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data.drop(['movieId', 'title', 'genres', 'tag', 'mean_rating'], axis=1)
y_train = train_data['mean_rating']
X_test = test_data.drop(['movieId', 'title', 'genres', 'tag', 'mean_rating'], axis=1)
y_test = test_data['mean_rating']

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)


## Оценка `RMSE` и процент ошибок

In [6]:
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test data: {rmse}')
max_rating = data['mean_rating'].max()
percentage_error = (rmse / max_rating) * 100
print(f'Процентная ошибка: {percentage_error:.2f}%')

RMSE on test data: 0.8518352980405883
Процентная ошибка: 17.04%


## Предсказание рейтингов для всех фильмов в датасете и вывод `5`

In [20]:
t = data.drop(['movieId', 'title', 'genres', 'tag', 'mean_rating'], axis=1)
t.columns = t.columns.astype(str)
all_movie_predictions = model.predict(t)

all_recommendations = pd.DataFrame({
    'movieId': data['movieId'],
    'title': data['title'],
    'tag': data['tag'],
    'genres': data['genres'],
    'mean_rating': data['mean_rating'],
    'predicted_rating': all_movie_predictions
})

all_recommendations = all_recommendations.sort_values(by='predicted_rating', ascending=False)

top_N = 5
print(f'Top {top_N} рекомендаций для нового пользователя:')
for i in range(top_N):
    movie_title = all_recommendations.iloc[i]['title']
    movie_tag = all_recommendations.iloc[i]['tag']
    movie_genres = all_recommendations.iloc[i]['genres']
    mean_rating = all_recommendations.iloc[i]['mean_rating']
    predicted_rating = all_recommendations.iloc[i]['predicted_rating']
    # print(f'{i + 1}. {movie_title:<50} - {mean_rating:^10} - {movie_genres:<30} - {movie_tag:<30} - Предсказанный рейтинг: {predicted_rating}')
    print(f'{i + 1}. {movie_title:<50} - рейтинг: {round(mean_rating, 3):^10} - Предсказанный рейтинг: {round(predicted_rating, 3)} - {movie_genres[:20]:<20} - {movie_tag[:20]:<10}... ')


Top 5 рекомендаций для нового пользователя:
1. More (1998)                                        - рейтинг:    5.0     - Предсказанный рейтинг: 4.724 - Animation|Drama|Sci- - claymation creativit... 
2. Going Places (Valseuses, Les) (1974)               - рейтинг:    5.0     - Предсказанный рейтинг: 4.692 - Comedy|Crime|Drama   - irreverent... 
3. Who Killed Chea Vichea? (2010)                     - рейтинг:    5.0     - Предсказанный рейтинг: 4.654 - Documentary          - Cambodia crime human... 
4. Lady Jane (1986)                                   - рейтинг:    5.0     - Предсказанный рейтинг: 4.626 - Drama|Romance        - England   ... 
5. Loving Vincent (2017)                              - рейтинг:    5.0     - Предсказанный рейтинг: 4.609 - Animation|Crime|Dram - NoTag     ... 


## Предложение фильмов для пользователя

In [36]:
new_user_data = pd.DataFrame({
    'tag': ['claymation creativit Cambodia crime action', 'MMA', 'action', 'romance'],
    'genres': ['Documentary|Romance', 'Romance', 'Action', 'Adventure']
})

all_tags = pd.concat([data['tag'], new_user_data['tag']])
tfidf_vectorizer_tags.fit_transform(all_tags)

new_user_tags_tfidf = tfidf_vectorizer_tags.transform(new_user_data['tag'])
new_user_genres_tfidf = tfidf_vectorizer_genres.transform(new_user_data['genres'])

new_user_tags_df = pd.DataFrame(new_user_tags_tfidf.toarray(), columns=tfidf_vectorizer_tags.get_feature_names_out())
new_user_genres_df = pd.DataFrame(new_user_genres_tfidf.toarray(), columns=tfidf_vectorizer_genres.get_feature_names_out())

new_user_data = pd.concat([new_user_data, new_user_tags_df, new_user_genres_df], axis=1)

new_user_data = new_user_data.drop(['tag', 'genres'], axis=1)
new_user_data = new_user_data.loc[:, ~new_user_data.columns.duplicated()]
columns_to_reindex = X_train.columns
new_user_data = new_user_data.reindex(columns=columns_to_reindex, fill_value=0)

new_predictions = model.predict(new_user_data)
new_predictions

selected_recommendations = all_recommendations.loc[
    all_recommendations['predicted_rating'].between(new_predictions[0] - 0.1, new_predictions[0] + 0.1)
]

selected_recommendations = selected_recommendations.sort_values(by='predicted_rating', ascending=False)

print(f'Top {top_N} рекомендаций для нового пользователя с рейтингом {new_predictions[0]}:')
for i in range(top_N):
    movie_title = selected_recommendations.iloc[i]['title']
    movie_tag = selected_recommendations.iloc[i]['tag']
    movie_genres = selected_recommendations.iloc[i]['genres']
    mean_rating = selected_recommendations.iloc[i]['mean_rating']
    predicted_rating = selected_recommendations.iloc[i]['predicted_rating']
    print(f'{i + 1}. {movie_title:<50} - рейтинг: {round(mean_rating, 3):^10} - Предсказанный рейтинг: {round(predicted_rating, 3)} - {movie_genres[:20]:<20} - {movie_tag[:20]:<10}... ')

Top 5 рекомендаций для нового пользователя с рейтингом 3.7987123848001723:
1. Moon (2009)                                        - рейтинг:   3.969    - Предсказанный рейтинг: 3.898 - Drama|Mystery|Sci-Fi - 2001-like Sci-fi sol... 
2. Hard-Boiled (Lat sau san taam) (1992)              - рейтинг:    4.0     - Предсказанный рейтинг: 3.898 - Action|Crime|Drama|T - gun fu heroic bloods... 
3. Guardians of the Galaxy (2014)                     - рейтинг:   4.051    - Предсказанный рейтинг: 3.898 - Action|Adventure|Sci - funny Great Visuals ... 
4. The Revenant (2015)                                - рейтинг:   3.903    - Предсказанный рейтинг: 3.898 - Adventure|Drama      - leonardo DiCarpio su... 
5. American Gangster (2007)                           - рейтинг:   3.905    - Предсказанный рейтинг: 3.897 - Crime|Drama|Thriller - corruption Denzel Wa... 


## Вывод

`Обработка данных:`
    + Данные о фильмах обогатил информацией о тегах, жанрах и средних рейтингах.
    + Применил `TF-IDF` для векторизации тегов и жанров.
`Создание модели:`
    + Использовал случайный лес для предсказания средних рейтингов фильмов.
`Оценка модели:`
    + `RMSE` on `test data`: `0.8518352980405883`
    + Процентная ошибка: `17.04%`
`Рекомендации для нового пользователя или какую оценку он бы поставил:`
    + Добавил предпочтения нового пользователя по тегам и жанрам.
    + так как по данным тегам и жанру его оценка составила `3,9` (для первого фильма) то он бы вероятно выставил рейтинг для данных фильмов в диапазоне `3.3` и `4.3`
